<a href="https://colab.research.google.com/github/flora0110/podcast_segment_headine/blob/main/similarity_ok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 42.7 MB/s 
     |████████████████████████████████| 1.2 MB 60.6 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 6.6 MB 55.9 MB/s 
     |████████████████████████████████| 895 kB 56.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=369d7eaffe17cd917457d7246ee3a251598d7cf99ca2be25fb284a06bf0b56b2
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer, util
#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
def similarity(podcastDict, sampleDict):
  print(len(sampleDict))
  arr = []
  #index = 0
  podcastN = len(podcastDict)-1
  sampleN = len(sampleDict)-1
  print(sampleN)
  for i in range(podcastN):
    #print(i)
    start = podcastDict[i]['time']
    end = podcastDict[i+1]['time']
    #print(arr)
    #print("start: "+str(start)+" end: "+str(end))
    for j in range(sampleN):
      #print("j: "+str(j))
      if(start>=sampleDict[j]['time'] and end<=sampleDict[j+1]['time']):
        #print("continue "+str(j))
        #print(sampleDict[j]['time'])
        #print(sampleDict[j+1]['time'])
        arr.append({'start_match':j,'ratio_start':(end-start)/(sampleDict[j+1]['time']-sampleDict[j]['time']),'end_match':-1,'ratio_end':-1})
        #print(len(arr))
        #index+=1
        break
      elif(start>=sampleDict[j]['time'] and start<=sampleDict[j+1]['time']):
        #print("not con "+str(j))
        #print(sampleDict[j]['time'])
        #print(sampleDict[j+1]['time'])
        arr.append({'start_match':j,'ratio_start':(sampleDict[j+1]['time']-start)/(sampleDict[j+1]['time']-sampleDict[j]['time'])})
        #print(len(arr))
        for k in range(j,sampleN):
          #print("kkkk "+str(k))
          #print(sampleDict[k]['time'])
          #print(sampleDict[k+1]['time'])
          if(end<=sampleDict[k+1]['time'] and end>=sampleDict[k]['time']):
            #print("update")
            arr[len(arr)-1].update({'end_match':k,'ratio_end':(end-sampleDict[k]['time'])/(sampleDict[k+1]['time']-sampleDict[k]['time'])})
            #print(len(arr))
            #index+=1
        break

  #print(arr)
  #for a in arr:
  #  print(a)
  print(len(podcastDict))
  print(len(sampleDict))
  print(len(arr))
  scores = []
  for i in range(0,len(arr)):
    #print(i)
    if(arr[i]['end_match']==-1):
      embedding_1= model.encode(podcastDict[i]['headline'], convert_to_tensor=True)
      embedding_2 = model.encode(sampleDict[arr[i]['start_match']]['headline'], convert_to_tensor=True)
      #print(podcastDict[i]['headline'])
      #print(sampleDict[arr[i]['start_match']]['headline'])
      score_temp = float(util.pytorch_cos_sim(embedding_1, embedding_2))
      score_temp*=arr[i]['ratio_start']
      scores.append(score_temp)
      #print(score_temp)
    else:

      embedding_1= model.encode(podcastDict[i]['headline'], convert_to_tensor=True)
      embedding_2 = model.encode(sampleDict[arr[i]['start_match']]['headline'], convert_to_tensor=True)
      score_temp = float(util.pytorch_cos_sim(embedding_1, embedding_2))
      score_temp*=arr[i]['ratio_start']
      score_temp_sum=score_temp
      for j in range(arr[i]['start_match']+1,arr[i]['end_match']):
        embedding_1= model.encode(podcastDict[i]['headline'], convert_to_tensor=True)
        embedding_2 = model.encode(sampleDict[j]['headline'], convert_to_tensor=True)
        score_temp = float(util.pytorch_cos_sim(embedding_1, embedding_2))
        score_temp_sum+=score_temp
      embedding_1= model.encode(podcastDict[i]['headline'], convert_to_tensor=True)
      embedding_2 = model.encode(sampleDict[j]['headline'], convert_to_tensor=True)
      score_temp = float(util.pytorch_cos_sim(embedding_1, embedding_2))
      score_temp*=arr[i]['ratio_end']
      score_temp_sum+=score_temp
      scores.append(score_temp_sum/(arr[i]['end_match']-arr[i]['start_match']+1))
  print(scores)
  sum = 0
  for score in scores:
    sum +=score
  print(sum/len(sampleDict))

# social

In [ ]:
sampleDict = [{'time': 0.0, 'headline': 'Social Bonding: Child-Parent, Romantic, Friendship, Breakups'}, {'time': 175.0, 'headline': 'ROKA, Athletic Greens, Headspace'}, {'time': 428.0, 'headline': 'Social Bonding as a Biological Process'}, {'time': 603.0, 'headline': 'Social Isolation'}, {'time': 812.0, 'headline': 'Social Homeostasis & Neural Circuits for Social Drive '}, {'time': 1135.0, 'headline': 'Brain Areas & (Neuro)Chemistry of Social Drive '}, {'time': 1368.0, 'headline': 'What is Social Homeostasis & Dopamine'}, {'time': 1620.0, 'headline': 'When We Lack Social Interactions: Short- Versus Long-Term'}, {'time': 1690.0, 'headline': 'Introverts & Extroverts'}, {'time': 1860.0, 'headline': '"Good" Versus "Bad" Social Interactions & Hierarchies'}, {'time': 2034.0, 'headline': 'Loneliness & Dorsal Raphe Nucleus & Social Hunger '}, {'time': 2253.0, 'headline': 'Tools'}, {'time': 2285.0, 'headline': 'Socializing & Food Appetite: Crossover Craving'}, {'time': 2565.0, 'headline': 'Falling in Love'}, {'time': 2705.0, 'headline': 'Tools for Social Bonds: Merging Physiologies; Story'}, {'time': 3234.0, 'headline': 'Childhood Attachment Patterns in Adulthood'}, {'time': 3825.0, 'headline': 'Attachment Styles: Autonomic Versus Intellectual Attachment'}, {'time': 3970.0, 'headline': 'Emotional Empathy & Cognitive Empathy, Arguing'}, {'time': 4185.0, 'headline': 'Allan N. Schore & "Right Brain Psychotherapy"'}, {'time': 4240.0, 'headline': 'Oxytocin & Trust, In Males Versus Females, Hormonal Glue'}, {'time': 4570.0, 'headline': 'Repairing Broken Bonds to Self & Others'}, {'time': 4736.0, 'headline': 'Social (Media) Butterflies: Biological Basis'}, {'time': 5048.0, 'headline': 'Key Points for Bonding & Understanding Social Bonds'}, {'time': 5227.0, 'headline': 'Breaking Up'}, {'time': 5316.0, 'headline': 'Synthesis '}, {'time': 5477.0, 'headline': 'Zero-Cost Support, Sponsors, Patreon, Thorne, Instagram, Twitter'},{'time':5596.9,'headline':""}]


#0.5 not dele

In [ ]:
#podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace Special Offer - Get Your Athletic Greens Today'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]
#podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace Special Offer - Get Your Athletic Greens Today'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]
#podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace Special Offer - Get Your Athletic Greens Today'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]
podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace Special Offer - Get Your Athletic Greens Today'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]
print(len(podcastDict))

46


In [ ]:
similarity(sampleDict,podcastDict)

46
45
27
46
25
[0.018658763423540958, 0.08208069683681998, 0.05339618959143983, 0.04056734047238806, 0.22446321726299787, 0.1864418546316589, 0.031053371264989846, 0.2577594544930074, 0.06420580514747933, 0.032529187047373634, 0.04070488827620514, -0.00026563274637221514, 0.09112313007580154, 0.134130311162976, 0.1611819586179194, 0.20082982758680878, 0.12691268144823983, 0.2306863266301242, -7.646688606712169e-05, 0.15384305223189215, 0.024315105637765382, 0.19057990236749858, 0.21076274676169304, 0.03396446962263871, 0.09295047452484308]
0.05832170990181875


## 0.5 dele spon

In [ ]:
podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace - The Best Deals'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]
print(len(podcastDict))

46


In [ ]:
similarity(sampleDict,podcastDict)

46
45
27
46
25
[0.018658763423540958, 0.08122123012422545, 0.05339618959143983, 0.04056734047238806, 0.22446321726299787, 0.1864418546316589, 0.031053371264989846, 0.2577594544930074, 0.06420580514747933, 0.032529187047373634, 0.04070488827620514, -0.00026563274637221514, 0.09112313007580154, 0.134130311162976, 0.1611819586179194, 0.20082982758680878, 0.12691268144823983, 0.2306863266301242, -7.646688606712169e-05, 0.15384305223189215, 0.024315105637765382, 0.19057990236749858, 0.21076274676169304, 0.03396446962263871, 0.09295047452484308]
0.05830302584284929


## 0.6 not dele

In [ ]:
podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Biology of Social Bonding'}, {'time': 1164.5, 'headline': ' The Social Homeostasis Circuit , a Neuronal Response to the Dop'}, {'time': 1369.0, 'headline': ' The Social Homeostasis Effect'}, {'time': 2727.5, 'headline': ' How to Enhance the Quality and Depth of Social Bonds'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Autonomic Nervous System of the Infant Mother'}, {'time': 3580.3, 'headline': ' Bonding in Children and Adults'}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' The Relationship Between Cognitive Empathy and Emotional Empathy'}, {'time': 4262.3, 'headline': ' The Effects of MDMA on the Body and Mind'}, {'time': 4823.5, 'headline': ' Social Bonding - What Are the Keys to Social Bonding?'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast - Andrew Huberman'},{'time':5596.9,'headline':""}]
print(len(podcastDict))

15


In [ ]:
similarity(sampleDict,podcastDict)

15
14
27
15
25
[0.14562761661979687, 0.013833481760587277, 0.1658694192621845, 0.12624794105387374, 0.04164150847254268, 0.009568685388004296, 0.04046132031898091, 0.046032916371946085, 0.019898537089818535, 0.056302319061769295, 0.010507354062402954, -0.03602597745938208, 0.035403221461717635, 0.04117410892592322, 0.11901714133497229, 0.13945982128879278, 0.22016500280586573, 0.1390637963871944, 0.09128128736351976, 0.04087723988086433, 0.023819854413129296, 0.0810125101232551, 0.11992376546293598, 0.02522587688485945, 0.14028362404907638]
0.12377815815897542


## 0.6 dele spon

In [ ]:
podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Biology of Social Bonding'}, {'time': 1164.5, 'headline': ' The Social Homeostasis Circuit , a Neuronal Response to the Dop'}, {'time': 1369.0, 'headline': ' The Social Homeostasis Effect'}, {'time': 2727.5, 'headline': ' How to Enhance the Quality and Depth of Social Bonds'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Autonomic Nervous System of the Infant Mother'}, {'time': 3580.3, 'headline': ' Bonding in Children and Adults'}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' The Relationship Between Cognitive Empathy and Emotional Empathy'}, {'time': 4262.3, 'headline': ' The Effects of MDMA on the Body and Mind'}, {'time': 4823.5, 'headline': ' Social Bonding - What Are the Keys to Social Bonding?'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast - Andrew Huberman'},{'time':5596.9,'headline':""}]
print(len(podcastDict))


15


In [ ]:
similarity(sampleDict,podcastDict)

15
14
27
15
25
[0.14562761661979687, 0.013833481760587277, 0.1658694192621845, 0.12624794105387374, 0.04164150847254268, 0.009568685388004296, 0.04046132031898091, 0.046032916371946085, 0.019898537089818535, 0.056302319061769295, 0.010507354062402954, -0.03602597745938208, 0.035403221461717635, 0.04117410892592322, 0.11901714133497229, 0.13945982128879278, 0.22016500280586573, 0.1390637963871944, 0.09128128736351976, 0.04087723988086433, 0.023819854413129296, 0.0810125101232551, 0.11992376546293598, 0.02522587688485945, 0.14028362404907638]
0.12377815815897542


## 0.7 no dele

In [ ]:
podcastDict = [{'time': 0.2, 'headline': ' The Biology of Social Bonding'}, {'time': 933.8, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1428.3, 'headline': ' The Social Homeostasis Circuit'}, {'time': 2727.5, 'headline': ' How to Enhance the Quality and Depth of Social Bonds'}, {'time': 3234.3, 'headline': ' The Relationship Between Parent and Child , And The Relationship Between Right Brain and Left Brain'}, {'time': 3873.9, 'headline': ' Cognitive Connections - How to Have a Deep Emotional Connection'}, {'time': 3917.5, 'headline': ' The Relationship Between Cognitive Empathy and Emotional Empathy'}, {'time': 4262.3, 'headline': ' The Effects of MDMA on the Body and Mind'}, {'time': 4823.5, 'headline': ' Social Bonding - What Are the Keys to Social Bonding?'}, {'time': 5482.1, 'headline': ' The Human Lab Podcast - Andrew Huberman'},{'time':5596.9,'headline':""}]
print(len(podcastDict))


11


In [ ]:
similarity(sampleDict,podcastDict)

11
10
27
11
25
[0.17276901046281815, 0.016411701368922952, 0.19678338557711605, 0.11398429959170356, 0.3923715142075526, 0.05499107993914762, 0.04149103408743595, 0.03781477218778262, 0.01753067265131644, 0.0544999157688741, 0.011226154606917808, -0.010619299678966916, 0.03397302894756712, 0.08051995408282317, 0.26950695780501155, 0.13995029398788514, 0.22016500280586573, 0.1390637963871944, 0.09818767723501645, 0.04087723988086433, 0.033047575904156704, 0.0801883622067263, 0.11870376967095034, 0.04938301564421363, 0.017917310921693075]
0.22006711147732624


## dele spon

In [ ]:
podcastDict = [{'time': 0.2, 'headline': ' The Biology of Social Bonding'}, {'time': 933.8, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1428.3, 'headline': ' The Social Homeostasis Circuit'}, {'time': 2727.5, 'headline': ' How to Enhance the Quality and Depth of Social Bonds'}, {'time': 3234.3, 'headline': ' The Relationship Between Parent and Child , And The Relationship Between Right Brain and Left Brain'}, {'time': 3873.9, 'headline': ' Cognitive Connections - How to Have a Deep Emotional Connection'}, {'time': 3917.5, 'headline': ' The Relationship Between Cognitive Empathy and Emotional Empathy'}, {'time': 4262.3, 'headline': ' The Effects of MDMA on the Body and Mind'}, {'time': 4823.5, 'headline': ' Social Bonding - What Are the Keys to Social Bonding?'}, {'time': 5482.1, 'headline': ' The Human Lab Podcast - Andrew Huberman'},{'time':5596.9,'headline':""}]
print(len(podcastDict))

11


In [ ]:
similarity(sampleDict,podcastDict)

11
10
27
11
25
[0.17276901046281815, 0.016411701368922952, 0.19678338557711605, 0.11398429959170356, 0.3923715142075526, 0.05499107993914762, 0.04149103408743595, 0.03781477218778262, 0.01753067265131644, 0.0544999157688741, 0.011226154606917808, -0.010619299678966916, 0.03397302894756712, 0.08051995408282317, 0.26950695780501155, 0.13995029398788514, 0.22016500280586573, 0.1390637963871944, 0.09818767723501645, 0.04087723988086433, 0.033047575904156704, 0.0801883622067263, 0.11870376967095034, 0.04938301564421363, 0.017917310921693075]
0.22006711147732624


# brain

In [ ]:
sampleDict = [{'time': 0.0, 'headline': '- Introduction'}, {'time': 86.0, 'headline': '- Andrew’s Background'}, {'time': 929.0, 'headline': '- How Andrew Turned His Life Around'}, {'time': 1427.0, 'headline': '- How Andrew Combines Neuroscience with His Past'}, {'time': 1603.0, 'headline': '- The Human Brain: Internal State & External State'}, {'time': 2106.0, 'headline': '- Neural Plasticity - Steering the Nervous System'}, {'time': 2545.0, 'headline': '- State of Flow - The Dopamine System'}, {'time': 3547.0, 'headline': '- Process of Internal Rewards'},{'time': 3955.0, 'headline': '- Studying Fear, Courage, and Resilience'}, {'time': 4563.0, 'headline': '- How to Deal with Problems of Motivation and Focus'}, {'time': 5081.0, 'headline': '- Panoramic Vision vs Focal Vision'}, {'time': 5445.0, 'headline': '- Methods of Focus and Deep Rest'}, {'time': 5690.0, 'headline': '- Optic Flow and EMDR'}, {'time': 5930.0, 'headline': '- Andrew’s Work in Addiction'}, {'time': 6426.0, 'headline': '- A Bio-Marker for Addicts to Avoid Relapse'}, {'time': 6576.0, 'headline': '- Neuroscience Perspective on Political Polarization'}, {'time': 7155.0, 'headline': '- The Importance of Internal Control'}, {'time': 7497.0, 'headline': '- Is There Hope for Us?'},{'time':7936.6,'headline':''}]
print(len(podcastDict))

11


##0.6

## not dele

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Origin of the Brain'}, {'time': 299.8, 'headline': ' Then I Become a Professional Skateboarder and I Love the Community'}, {'time': 427.9, 'headline': ' Palo Alto Gunn High School - The Infamous High School'}, {'time': 479.8, 'headline': ' The Power of Mindfulness'}, {'time': 1616.4, 'headline': ' The Brain and Nervous System'}, {'time': 1710.3, 'headline': ' The Importance of Sensations in the Body'}, {'time': 2106.4, 'headline': " The Brain's Ability to Adapt to Experience"}, {'time': 2420.0, 'headline': ' Dopamine and the Growth Mindset'}, {'time': 3108.2, 'headline': " If it's Podcasting Great , If it's Building a Business Great If"}, {'time': 3119.8, 'headline': ' The Power of Dopamine'}, {'time': 3497.2, 'headline': ' The Selection Process Is Not Enough'}, {'time': 3914.9, 'headline': ' The Neuroscience of the Brain'}, {'time': 4626.8, 'headline': " Rapid , and we're actually running a human study on this now . "}, {'time': 4636.3, 'headline': ' How to Control Your Autonomic Nervous System'}, {'time': 5003.2, 'headline': ' The Visual Focus Is What Brings the Mind to the Brain'}, {'time': 5210.6, 'headline': ' How to Recover From Stress'}, {'time': 5679.6, 'headline': ' The Effect of Eye Movements on Addiction'}, {'time': 6012.5, 'headline': ' The Visual System and the Addiction Tree'}, {'time': 6319.0, 'headline': ' Is There a Way to Control Your Autonomic Nervous System?'}, {'time': 6709.5, 'headline': ' The Human Brain Deal , Deep into the Brain'}, {'time': 6763.1, 'headline': ' The Key to a Successful Addiction Treatment Program is to Listen to the Brain and Listen to'}, {'time': 7131.3, 'headline': ' The Next Generation of Addiction'}, {'time': 7864.9, 'headline': " I'm Doing It , You Paid For It You're Not a"},{'time':7936.6,'headline':''}]
print(len(podcastDict))

24


In [ ]:
similarity(sampleDict,podcastDict)

24
23
19
24
17
[0.09909526454134628, 0.022159261022962017, 0.02104208059510609, 0.23791098600326796, 0.14898699032062707, 0.004103383659765672, 0.05709460300261242, -0.018745994179381765, 0.1382128334895394, 0.17231837768424166, -0.006053520878253477, 0.1576495192078867, 0.11284009176530313, 0.04068149227491567, 0.1990290672186749, 0.08542324681415948, 0.12488923248842017]
0.06652653812629973


## dele spon

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Brain and the Neuroscience Podcast'}, {'time': 299.8, 'headline': ' Then I Become a Professional Skateboarder and I Love the Community'}, {'time': 427.9, 'headline': ' Palo Alto Gunn High School - The Infamous High School'}, {'time': 479.8, 'headline': ' The Power of Mindfulness'}, {'time': 1616.4, 'headline': ' The Brain and Nervous System'}, {'time': 1710.3, 'headline': ' The Importance of Sensations in the Body'}, {'time': 2106.4, 'headline': " The Brain's Ability to Adapt to Experience"}, {'time': 2420.0, 'headline': ' Dopamine and the Growth Mindset'}, {'time': 3108.2, 'headline': " If it's Podcasting Great , If it's Building a Business Great If"}, {'time': 3119.8, 'headline': ' The Power of Dopamine'}, {'time': 3497.2, 'headline': ' The Selection Process Is Not Enough'}, {'time': 3914.9, 'headline': ' The Neuroscience of the Brain'}, {'time': 4626.8, 'headline': " Rapid , and we're actually running a human study on this now . "}, {'time': 4636.3, 'headline': ' How to Control Your Autonomic Nervous System'}, {'time': 5003.2, 'headline': ' The Visual Focus Is What Brings the Mind to the Brain'}, {'time': 5210.6, 'headline': ' How to Recover From Stress'}, {'time': 5679.6, 'headline': ' The Effect of Eye Movements on Addiction'}, {'time': 6012.5, 'headline': ' The Visual System and the Addiction Tree'}, {'time': 6319.0, 'headline': ' Is There a Way to Control Your Autonomic Nervous System?'}, {'time': 6709.5, 'headline': ' The Human Brain Deal , Deep into the Brain'}, {'time': 6763.1, 'headline': ' The Key to a Successful Addiction Treatment Program is to Listen to the Brain and Listen to'}, {'time': 7131.3, 'headline': ' The Next Generation of Addiction'}, {'time': 7864.9, 'headline': " I'm Doing It , You Paid For It You're Not a"},{'time':7936.6,'headline':''}]
print(len(podcastDict))

24


In [ ]:
similarity(sampleDict,podcastDict)

24
23
19
24
17
[0.11252932264199765, 0.022159261022962017, 0.02104208059510609, 0.23791098600326796, 0.14898699032062707, 0.004103383659765672, 0.05709460300261242, -0.018745994179381765, 0.1382128334895394, 0.17231837768424166, -0.006053520878253477, 0.1576495192078867, 0.11284009176530313, 0.04068149227491567, 0.1990290672186749, 0.08542324681415948, 0.12488923248842017]
0.06708629054716021


## 0.7 not dele

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Origin of Skateboarding'}, {'time': 429.2, 'headline': ' Palo Alto Gunn High School - I Got Removed From High School'}, {'time': 479.8, 'headline': ' The Power of Mindfulness'}, {'time': 1616.4, 'headline': ' The Brain and Nervous System'}, {'time': 1710.3, 'headline': ' The Importance of Sensations in the Body'}, {'time': 2106.4, 'headline': ' The Dopamine System and the Growth Mindset'}, {'time': 3108.2, 'headline': " If it's Podcasting Great , If it's Building a Business Great If"}, {'time': 3119.8, 'headline': ' The Power of Dopamine'}, {'time': 3497.2, 'headline': ' The Selection Process Is Not Enough'}, {'time': 3914.9, 'headline': ' The Secret of the Neuronal Brain'}, {'time': 5003.2, 'headline': ' The Visual Focus Is What Brings the Mind to the Brain'}, {'time': 5210.6, 'headline': ' How to Recover From Addiction'}, {'time': 6217.8, 'headline': ' Is There a Future for Addiction Treatment?'}, {'time': 6709.5, 'headline': ' The Next Generation of Addiction and Neuroplasticity'},{'time':7936.6,'headline':''}]
print(len(podcastDict))

15


In [ ]:
similarity(sampleDict,podcastDict)

15
14
19
15
17
[0.06500269743338932, 0.022159261022962017, 0.02104208059510609, 0.23791098600326796, 0.10876618291447837, 0.006641148350180645, 0.051778125303359734, 0.03855349221650615, 0.065877548089264, 0.13063756531462556, -0.01240681884240735, 0.034944091740821075, 0.00431123942276989, 0.16113728412118467, 0.11177689100352299, 0.09365165717404952, 0.08168352789288583]
0.08156446398373111


## dele spon

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Brain and the Brain'}, {'time': 429.2, 'headline': ' Palo Alto Gunn High School - I Got Removed From High School'}, {'time': 479.8, 'headline': ' The Power of Mindfulness'}, {'time': 1616.4, 'headline': ' The Brain and Nervous System'}, {'time': 1680.3, 'headline': " The Brain and the Brain - The Brain's Ability to Adapt to the Environment"}, {'time': 2523.9, 'headline': ' Dopamine Releases the Dopamine System and How to Restore It'}, {'time': 3497.2, 'headline': ' The Selection Process Is Not Enough'}, {'time': 3914.9, 'headline': ' The Secret of the Neuronal Brain'}, {'time': 5003.2, 'headline': ' The Visual Focus Is What Brings the Mind to the Brain'}, {'time': 5210.6, 'headline': ' How to Recover From Addiction'}, {'time': 6217.8, 'headline': ' Is There a Future for Addiction Treatment?'}, {'time': 6709.5, 'headline': ' The Next Generation of Addiction and Neuroplasticity'},{'time':7936.6,'headline':''}]
print(len(podcastDict))

13


In [ ]:
similarity(sampleDict,podcastDict)

13
12
19
13
17
[0.07743612951592357, 0.022159261022962017, 0.02104208059510609, 0.17928570084002707, 0.11423096050421165, 0.06535770872960973, 0.047823952485722306, 0.03855349221650615, 0.07468246820527057, 0.13899771567281768, -0.01240681884240735, 0.034944091740821075, 0.024633927001196647, 0.16113728412118467, 0.09674173091120193, 0.09365165717404952, 0.08168352789288583]
0.09691960536823761


## 0.5

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' Neuroscientist Dr Andrew Huberman , Welcome to the Podcast'}, {'time': 84.2, 'headline': ' The Life of a Skateboarder'}, {'time': 299.8, 'headline': ' Then I Become a Professional Skateboarder and I Love the Community'}, {'time': 427.9, 'headline': ' Palo Alto Gunn High School'}, {'time': 466.6, 'headline': ' The Power of Concentration'}, {'time': 1174.1, 'headline': ' Learning How to Speak Properly'}, {'time': 1183.4, 'headline': ' The Power of Mindfulness'}, {'time': 1616.4, 'headline': ' The Brain and Nervous System'}, {'time': 1710.3, 'headline': ' Sensations , Perception Thoughts and Behaviors'}, {'time': 1797.7, 'headline': ' The Nervous System in Iraq'}, {'time': 1866.1, 'headline': " Impatience Is Not a Problem When You're In Line At The Store"}, {'time': 1938.5, 'headline': " The Brain's Ability to Adapt to the Environment"}, {'time': 2563.3, 'headline': " 'It's not a good thing , ' he said . "}, {'time': 2571.0, 'headline': ' The State of the State'}, {'time': 2715.9, 'headline': " I've Got an Hour . I'm Going to Write Beautifully for an Hour or"}, {'time': 2761.8, 'headline': ' Dopamine System - The Dopamine System'}, {'time': 2839.0, 'headline': ' Dopamine Release - Is It a Good Idea?'}, {'time': 2894.2, 'headline': ' Dopamine Released in Addictions'}, {'time': 2920.2, 'headline': ' The Growth Mindset'}, {'time': 3108.2, 'headline': " If it's Podcasting Great , If it's Building a Business Great If"}, {'time': 3119.8, 'headline': ' The Power of Dopamine'}, {'time': 3413.9, 'headline': ' Dopamine Suspressing Norepinephrine - Is It'}, {'time': 3443.2, 'headline': ' The Selection Process Is Not Enough'}, {'time': 3815.1, 'headline': ' TPO Cysts - Cold Water and Sleep Deprivation'}, {'time': 3843.9, 'headline': ' Sleep Deprivation and the Autonomic Nervous System'}, {'time': 3914.9, 'headline': " The Ultimate Determinant Isn't Your Physical Conditioning , But Your Resilience"}, {'time': 4114.5, 'headline': ' The Holy Grail of Neuroscience'}, {'time': 4320.4, 'headline': " The Brain's Control Panel for Addiction Behavior"}, {'time': 4456.7, 'headline': ' The Moving Forward Response to Stress'}, {'time': 4626.8, 'headline': " Rapid , and we're actually running a human study on this now . "}, {'time': 4636.3, 'headline': ' How to Control Your Autonomic Nervous System'}, {'time': 4777.1, 'headline': ' The Secret of Breathing'}, {'time': 4904.3, 'headline': ' The Power of Focus'}, {'time': 5003.2, 'headline': ' The Visual Focus Is What Brings the Mind to the Brain'}, {'time': 5210.6, 'headline': ' How to Reset Your Focus in Wakefulness'}, {'time': 4081.0, 'headline': " 'Tis the Season to Do Things"}, {'time': 5397.8, 'headline': ' Hypnosis Scripts - How to Turn Off Your Brain and Fall Asle'}, {'time': 5673.9, 'headline': ' EMDR - The Brain State of Stress'}, {'time': 5783.2, 'headline': ' The Effect of Eye Movements on Addiction'}, {'time': 5982.0, 'headline': ' Stress and Stress and Stress and Stress and Stress and Stress and Stress and Stress and Stress and Stress'}, {'time': 6012.5, 'headline': ' The Visual System and the Addiction Tree'}, {'time': 6319.0, 'headline': ' Is There a Way to Control Your Autonomic Nervous System?'}, {'time': 6709.5, 'headline': ' The Human Brain Deal , Deep into the Brain'}, {'time': 6763.1, 'headline': ' The Key to Recovery From Addiction'}, {'time': 7007.9, 'headline': ' We Love the Way We Feel'}, {'time': 7018.2, 'headline': ' The Next Generation of Kids Need to Learn to Regulate Their Nervous System'}, {'time': 7131.3, 'headline': ' The Next Generation of Neuroplasticity'}, {'time': 7637.4, 'headline': '  - I think that , once people start to realize that your high performing military military'}, {'time': 7647.7, 'headline': ' The Next Generation of Addiction and Trauma'}, {'time': 7870.9, 'headline': " I'm Doing It , You Paid For It You Did It You Pai"},{'time':7936.6,'headline':''}]
print(len(podcastDict))

51


In [ ]:
similarity(sampleDict,podcastDict)

51
50
19
51
17
[0.08792662706843596, 0.026688500271737706, 0.05523424666142629, 0.127169627049365, 0.3023514862287975, 0.0404218356651204, 0.136372970525178, 0.060694151311080156, 0.22632934798300497, 0.21870639917870038, -0.007531029486369583, 0.1361338063902218, 0.10639754202723152, 0.04068149227491567, 0.17935666152334004, 0.25831836811805126, 0.11600187652267924]
0.04139713547672384


## dele spon

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' Neuroscientist Dr Andrew Huberman , Welcome to the Podcast'}, {'time': 84.2, 'headline': ' The History of Chaos Theory'}, {'time': 299.8, 'headline': ' Then I Become a Professional Skateboarder and I Love the Community'}, {'time': 427.9, 'headline': ' Palo Alto Gunn High School'}, {'time': 466.6, 'headline': ' The Power of Concentration'}, {'time': 1174.1, 'headline': ' Learning How to Speak Properly'}, {'time': 1183.4, 'headline': ' The Power of Mindfulness'}, {'time': 1616.4, 'headline': ' The Brain and Nervous System'}, {'time': 1710.3, 'headline': ' Sensations , Perception Thoughts and Behaviors'}, {'time': 1797.7, 'headline': ' The Nervous System in Iraq'}, {'time': 1866.1, 'headline': " Impatience Is Not a Problem When You're In Line At The Store"}, {'time': 1938.5, 'headline': " The Brain's Ability to Adapt to the Environment"}, {'time': 2563.3, 'headline': " 'It's not a good thing , ' he said . "}, {'time': 2571.0, 'headline': ' The State of the State'}, {'time': 2715.9, 'headline': " I've Got an Hour . I'm Going to Write Beautifully for an Hour or"}, {'time': 2761.8, 'headline': ' Dopamine System - The Dopamine System'}, {'time': 2839.0, 'headline': ' Dopamine Release - Is It a Good Idea?'}, {'time': 2894.2, 'headline': ' Dopamine Released in Addictions'}, {'time': 2920.2, 'headline': ' The Growth Mindset'}, {'time': 3108.2, 'headline': " If it's Podcasting Great , If it's Building a Business Great If"}, {'time': 3119.8, 'headline': ' The Power of Dopamine'}, {'time': 3413.9, 'headline': ' Dopamine Suspressing Norepinephrine - Is It'}, {'time': 3443.2, 'headline': ' The Selection Process Is Not Enough'}, {'time': 3815.1, 'headline': ' TPO Cysts - Cold Water and Sleep Deprivation'}, {'time': 3843.9, 'headline': ' Sleep Deprivation and the Autonomic Nervous System'}, {'time': 3914.9, 'headline': " The Ultimate Determinant Isn't Your Physical Conditioning , But Your Resilience"}, {'time': 4114.5, 'headline': ' The Holy Grail of Neuroscience'}, {'time': 4320.4, 'headline': " The Brain's Control Panel for Addiction Behavior"}, {'time': 4456.7, 'headline': ' The Moving Forward Response to Stress'}, {'time': 4626.8, 'headline': " Rapid , and we're actually running a human study on this now . "}, {'time': 4636.3, 'headline': ' How to Control Your Autonomic Nervous System'}, {'time': 4777.1, 'headline': ' The Secret of Breathing'}, {'time': 4904.3, 'headline': ' The Power of Focus'}, {'time': 5003.2, 'headline': ' The Visual Focus Is What Brings the Mind to the Brain'}, {'time': 5210.6, 'headline': ' How to Reset Your Focus in Wakefulness'}, {'time': 4081.0, 'headline': " 'Tis the Season to Do Things"}, {'time': 5397.8, 'headline': ' Hypnosis Scripts - How to Turn Off Your Brain and Fall Asle'}, {'time': 5673.9, 'headline': ' EMDR - The Brain State of Stress'}, {'time': 5783.2, 'headline': ' The Effect of Eye Movements on Addiction'}, {'time': 5982.0, 'headline': ' Stress and Stress and Stress and Stress and Stress and Stress and Stress and Stress and Stress and Stress'}, {'time': 6012.5, 'headline': ' The Visual System and the Addiction Tree'}, {'time': 6319.0, 'headline': ' Is There a Way to Control Your Autonomic Nervous System?'}, {'time': 6709.5, 'headline': ' The Human Brain Deal , Deep into the Brain'}, {'time': 6763.1, 'headline': ' The Key to Recovery From Addiction'}, {'time': 7007.9, 'headline': ' We Love the Way We Feel'}, {'time': 7018.2, 'headline': ' The Next Generation of Kids Need to Learn to Regulate Their Nervous System'}, {'time': 7131.3, 'headline': ' The Next Generation of Neuroplasticity'}, {'time': 7637.4, 'headline': '  - I think that , once people start to realize that your high performing military military'}, {'time': 7647.7, 'headline': ' The Next Generation of Addiction and Trauma'}, {'time': 7870.9, 'headline': " I'm Doing It , You Paid For It You Did It You Pai"},{'time':7936.6,'headline':''}]
print(len(podcastDict))

51


In [ ]:
similarity(sampleDict,podcastDict)

51
50
19
51
17
[0.15945433595309977, 0.026688500271737706, 0.05523424666142629, 0.127169627049365, 0.3023514862287975, 0.0404218356651204, 0.136372970525178, 0.060694151311080156, 0.22632934798300497, 0.21870639917870038, -0.007531029486369583, 0.1361338063902218, 0.10639754202723152, 0.04068149227491567, 0.17935666152334004, 0.25831836811805126, 0.11600187652267924]
0.04279963957250157


# sugar

In [ ]:
sampleDict = [{'time': 0.0, 'headline': 'Sugar & Physiology   '}, {'time': 145.0, 'headline': 'The Brain-Body Contract '}, {'time': 193.0, 'headline': 'Thesis, AG1 (Athletic Greens), InsideTracker '}, {'time': 460.0, 'headline': 'Sugar & the Brain '}, {'time': 606.0, 'headline': 'Appetite & Hormones: Ghrelin & Insulin '}, {'time': 857.0, 'headline': 'Glucose & Brain Function'}, {'time': 1459.0, 'headline': 'Glucose & Physical Activity '}, {'time': 1576.0, 'headline': 'Fructose vs. Glucose '}, {'time': 1961.0, 'headline': 'When to Eat High-Sugar Foods?  '}, {'time': 2101.0, 'headline': 'Sugar’s Taste vs. Nutritive Pathways, Sugar Cravings  '}, {'time': 2506.0, 'headline': 'Tool: Sugar & the Dopamine, Pleasure - Pain Dichotomy  '}, {'time': 2923.0, 'headline': 'Subconscious Sugar Circuits, Hidden Sugars in Food '}, {'time': 3483.0, 'headline': 'Glucose Metabolism in the Brain '}, {'time': 3780.0, 'headline': 'Tool: Glycemic Index, Blunting Sugar Cravings   '}, {'time': 4328.0, 'headline': 'Sugary Drinks, Highly Refined Sugars '}, {'time': 4473.0, 'headline': 'Artificial Sweeteners'}, {'time': 4956.0, 'headline': 'ADHD, Omega-3s '}, {'time': 5418.0, 'headline': 'Tools: Reduce Sugar Cravings with EPA Omega-3s & Glutamine'}, {'time': 5715.0, 'headline': 'Tool: Blunt Sugar Peaks & Craving with Lemon Juice'}, {'time': 6189.0, 'headline': 'Tool: Reduce Sugar Cravings & Spikes with Cinnamon  '}, {'time': 6310.0, 'headline': 'Berberine, Sustained Low Blood Glucose Levels'}, {'time': 6684.0, 'headline': 'Tool: Sleep & Sugar Cravings'}, {'time': 6993.0, 'headline': 'Zero-Cost Support, YouTube Feedback, Spotify, Apple Reviews, Sponsors, Patreon, Instagram, Twitter, Thorne, Neural Network Newsletter'},{'time':7131.0,'headline':''}]

## 0.7 not dele

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Brain and the Brain - The Science and Science Podcast'}, {'time': 675.9, 'headline': ' Type 1 Diabetes - How to Clamp Blood Glucose Levels'}, {'time': 782.2, 'headline': ' The Importance of Glucose in the Nervous System'}, {'time': 1595.4, 'headline': ' Is Fructose a Good Thing?'}, {'time': 2076.4, 'headline': ' Dopamine and the Pleasure Pain - The Secret of the Mood and'}, {'time': 2983.2, 'headline': ' The Preference for Sweet Taste in Foods'}, {'time': 62.4, 'headline': ' The Effects of Sugary Foods on Your Blood Sugar'}, {'time': 4478.6, 'headline': ' Is There a Conversion of Artificial Sweeteners?'}, {'time': 4749.8, 'headline': ' Is This a Fair Experiment?'}, {'time': 4796.1, 'headline': ' ADHD and the Effects of Sugar Consumption'}, {'time': 5656.2, 'headline': ' Can Lemon Juice And Lime Juice Reduce Blood Glucose?'}, {'time': 6656.8, 'headline': ' The Importance of Sleep in Controlling Appetite'},{'time':7131.0,'headline':''}]
print(len(podcastDict))

13


In [ ]:
similarity(sampleDict,podcastDict)

13
12
24
13
22
[0.00813933405236896, 0.18275988356853315, 0.029590832927799255, 0.26702019860875625, 0.2658997775150295, 0.10518458123519041, 0.09884381798928084, 0.1568169289768871, 0.040875466677668774, 0.1538194188158787, 0.17307717401676634, 0.03925139133299958, 0.046541421052030844, 0.015878926148249838, 0.12863252530692662, 0.17075273938240565, 0.0686439185349002, 0.17975376131485485, 0.05688130408793525, 0.05985877058611977, 0.06366020658507927, 0.16802900303068774]
0.1907624139804884


## dele spon

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Brain and the Body - The Brain and the Brain'}, {'time': 675.9, 'headline': ' Type 1 Diabetes - How to Clamp Blood Glucose Levels'}, {'time': 782.2, 'headline': ' The Importance of Glucose in the Nervous System'}, {'time': 1595.4, 'headline': ' Is Fructose a Good Thing?'}, {'time': 2076.4, 'headline': ' Dopamine and the Pleasure Pain - The Secret of the Mood and'}, {'time': 2983.2, 'headline': ' The Preference for Sweet Taste in Foods'}, {'time': 62.4, 'headline': ' The Effects of Sugary Foods on Your Blood Sugar'}, {'time': 4478.6, 'headline': ' Is There a Conversion of Artificial Sweeteners?'}, {'time': 4749.8, 'headline': ' Is This a Fair Experiment?'}, {'time': 4796.1, 'headline': ' ADHD and the Effects of Sugar Consumption'}, {'time': 5656.2, 'headline': ' Can Lemon Juice And Lime Juice Reduce Blood Glucose?'}, {'time': 6656.8, 'headline': ' The Importance of Sleep in Controlling Appetite'},{'time':7131.0,'headline':''}]
print(len(podcastDict))

13


In [ ]:
for p in podcastDict:
  print(p['headline'])

 The Brain and the Body - The Brain and the Brain
 Type 1 Diabetes - How to Clamp Blood Glucose Levels
 The Importance of Glucose in the Nervous System
 Is Fructose a Good Thing?
 Dopamine and the Pleasure Pain - The Secret of the Mood and
 The Preference for Sweet Taste in Foods
 The Effects of Sugary Foods on Your Blood Sugar
 Is There a Conversion of Artificial Sweeteners?
 Is This a Fair Experiment?
 ADHD and the Effects of Sugar Consumption
 Can Lemon Juice And Lime Juice Reduce Blood Glucose?
 The Importance of Sleep in Controlling Appetite



In [ ]:
similarity(sampleDict,podcastDict)

13
12
24
13
22
[0.011061036258662513, 0.24736743578017553, 0.010257389437220935, 0.26768553021454206, 0.2658997775150295, 0.10518458123519041, 0.09884381798928084, 0.1568169289768871, 0.040875466677668774, 0.1538194188158787, 0.17307717401676634, 0.03925139133299958, 0.046541421052030844, 0.015878926148249838, 0.12863252530692662, 0.17075273938240565, 0.0686439185349002, 0.17975376131485485, 0.05688130408793525, 0.05985877058611977, 0.06366020658507927, 0.16802900303068774]
0.19452096340611486


## 0.6 not dele

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Sweet Taste of Sugar and How It Impacts Your Brain and Body'}, {'time': 121.0, 'headline': ' The Brain Body Contract - Live Event May 1st'}, {'time': 184.9, 'headline': ' The Nootropics Podcast - The Best of the Best'}, {'time': 488.0, 'headline': ' The Effects of Sugar on the Nervous System'}, {'time': 690.4, 'headline': ' Glucose is Complete Blood Sugar'}, {'time': 718.1, 'headline': ' Diabetes - Is It a Good Idea to Avoid Diabetes?'}, {'time': 875.4, 'headline': " Ketogenic , I'm a Ketoholic I'm a Ke"}, {'time': 912.4, 'headline': ' Orientation Tuning in the Visual System'}, {'time': 1000.6, 'headline': ' Is It Dependent On Blood Glucose Level?'}, {'time': 1249.9, 'headline': ' Glucose is the Preferable Source of Fuel for the Nervous System'}, {'time': 1595.4, 'headline': ' Is Fructose a Good Thing for Weight Loss?'}, {'time': 2101.0, 'headline': ' Dopamine and the Pleasure Pain - The Secret of the Mood and'}, {'time': 3000.4, 'headline': ' The Preference for Sweet Taste in Foods'}, {'time': 62.4, 'headline': ' The Secret of the Ketogenic Diet'}, {'time': 4113.0, 'headline': ' Drug Cocaine Causes Sharp Increases in Dopamine , Within the Brain'}, {'time': 4167.0, 'headline': ' The New Evidence for Type 2 Diabetes'}, {'time': 4478.6, 'headline': ' Is There a Conversion of Artificial Sweeteners?'}, {'time': 4749.8, 'headline': ' Is That a Fair Experiment Or Is It Not Our Natural World Experi'}, {'time': 4781.2, 'headline': ' ADHD - Is Sugar Consumption a Good Idea?'}, {'time': 5128.7, 'headline': ' Sugar , Consumption of Sugar - Sugar Beverages and'}, {'time': 5157.7, 'headline': ' Can Glutamine Reduce Sugar Cravings?'}, {'time': 5717.2, 'headline': ' The Secret of Blood Glucose Monitoring'}, {'time': 6012.6, 'headline': ' Lemon Juice and Lime Juice Can Reduce Blood Sugar'}, {'time': 6207.0, 'headline': ' The Best Way to Lower Blood Glucose Is Toxic Cinnamon'}, {'time': 6443.1, 'headline': ' Increasing Blood Sugar Through Berberine'}, {'time': 6656.8, 'headline': ' The Role of Sleep in Controlling Sugar Cravings'}, {'time': 6941.8, 'headline': ' The Government Lab Podcast - Is It Worth It?'},{'time':7131.0,'headline':''}]
print(len(podcastDict))

28


In [ ]:
similarity(sampleDict,podcastDict)

28
27
24
28
22
[0.2766183718108807, -0.03663562142349477, 0.010272212410804678, 0.2839935172858208, 0.17611554083916092, 0.18277545741256515, 0.11287360790219239, 0.17234671955244452, 0.26422229400287656, 0.15508499998025213, 0.18403394401329312, 0.026820052567984757, 0.13655759434540318, 0.12122194147614966, 0.13827987775820752, 0.23424157204136467, -0.079651783522588, 0.19751629714835525, 0.07497772254297673, 0.28547911901078654, 0.22785078613934312, 0.03066019810751934]
0.1134162293357964


## dele spon

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Huberman Lab Podcast - The Brain Body Contract'}, {'time': 488.0, 'headline': ' The Effects of Sugar on the Nervous System'}, {'time': 690.4, 'headline': ' Type 1 Diabetes - Insulin Response to Clamp Blood Glucose Level'}, {'time': 782.2, 'headline': ' The Effects of Glucose on Brain Function'}, {'time': 912.4, 'headline': ' Orientation Tuning in the Visual System'}, {'time': 1000.6, 'headline': ' Is It Dependent On Blood Glucose Level?'}, {'time': 1249.9, 'headline': ' Glucose is the Preferable Source of Fuel for the Nervous System'}, {'time': 1595.4, 'headline': ' Is Fructose a Good Thing for Weight Loss?'}, {'time': 2101.0, 'headline': ' Dopamine and the Pleasure Pain - The Secret of the Mood and'}, {'time': 3000.4, 'headline': ' The Preference for Sweet Taste in Foods'}, {'time': 62.4, 'headline': ' The Effects of Sugary Foods on Your Blood Sugar'}, {'time': 4478.6, 'headline': ' Is There a Conversion of Artificial Sweeteners?'}, {'time': 4749.8, 'headline': ' Is That a Fair Experiment Or Is It Not Our Natural World Experi'}, {'time': 4781.2, 'headline': ' ADHD - Is Sugar Consumption a Good Idea?'}, {'time': 5128.7, 'headline': ' Sugar , Consumption of Sugar - Sugar Beverages and'}, {'time': 5157.7, 'headline': ' Can Glutamine Reduce Sugar Cravings?'}, {'time': 5717.2, 'headline': ' How Lemon Juice and Lime Juice Can Decrease Blood Glucose'}, {'time': 6218.6, 'headline': ' The Effects of Berberine on Blood Glucose'}, {'time': 6656.8, 'headline': ' The Role of Sleep in Controlling Sugar Cravings'}, {'time': 6941.8, 'headline': ' The Government Lab Podcast - Is It Worth It?'},{'time':7131.0,'headline':''}]
print(len(podcastDict))

21


In [ ]:
for p in podcastDict:
  print(p['headline'])

 The Huberman Lab Podcast - The Brain Body Contract
 The Effects of Sugar on the Nervous System
 Type 1 Diabetes - Insulin Response to Clamp Blood Glucose Level
 The Effects of Glucose on Brain Function
 Orientation Tuning in the Visual System
 Is It Dependent On Blood Glucose Level?
 Glucose is the Preferable Source of Fuel for the Nervous System
 Is Fructose a Good Thing for Weight Loss?
 Dopamine and the Pleasure Pain - The Secret of the Mood and
 The Preference for Sweet Taste in Foods
 The Effects of Sugary Foods on Your Blood Sugar
 Is There a Conversion of Artificial Sweeteners?
 Is That a Fair Experiment Or Is It Not Our Natural World Experi
 ADHD - Is Sugar Consumption a Good Idea?
 Sugar , Consumption of Sugar - Sugar Beverages and
 Can Glutamine Reduce Sugar Cravings?
 How Lemon Juice and Lime Juice Can Decrease Blood Glucose
 The Effects of Berberine on Blood Glucose
 The Role of Sleep in Controlling Sugar Cravings
 The Government Lab Podcast - Is It Worth It?



In [ ]:
similarity(sampleDict,podcastDict)

21
20
24
21
22
[0.005327329256417786, 0.33583526436386846, 0.006812431314892924, 0.30771431335165367, 0.19477075437673233, 0.18277545741256515, 0.11223195383803822, 0.17234671955244452, 0.25670441998298144, 0.15508499998025213, 0.17785003524785284, 0.03925139133299958, 0.046541421052030844, 0.015878926148249838, 0.13722190154525749, 0.23424157204136467, -0.079651783522588, 0.17283049535507794, 0.04463980448229888, 0.11566994061769716, 0.15514652740808998, 0.03066019810751934]
0.13428019396408084


## 0.5 not de

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Ins and Outs of Sugar and Other Foods Today'}, {'time': 147.4, 'headline': ' The Brain Body Contract - Live Event in Seattle , Washington on May 17th and Portland '}, {'time': 184.9, 'headline': ' The Nootropics Podcast - The Best of the Best'}, {'time': 488.0, 'headline': ' The Effects of Sugar on the Nervous System'}, {'time': 690.4, 'headline': ' Glucose is Complete Blood Sugar'}, {'time': 718.1, 'headline': ' Diabetes - Is It a Good Idea to Avoid Diabetes?'}, {'time': 875.4, 'headline': " Ketogenic , I'm a Ketoholic I'm a Ke"}, {'time': 912.4, 'headline': ' Orientation Tuning in the Visual System'}, {'time': 1000.6, 'headline': ' Is It Dependent On Blood Glucose Level?'}, {'time': 1249.9, 'headline': ' Glucose is the Preferable Source of Fuel for the Nervous System'}, {'time': 1595.4, 'headline': ' The Effects of Fructose in Fruits'}, {'time': 1739.7, 'headline': ' Is Fructose a Good Thing For Your Appetite?'}, {'time': 1962.8, 'headline': ' Sugar Is So Attractive'}, {'time': 2074.3, 'headline': ' The Parallel Pathways in the Brain and Body Control Sugar Cravings'}, {'time': 2360.4, 'headline': ' Neuroimaging Study - When You Inhale Something Sweet , You Taste It'}, {'time': 2454.6, 'headline': ' Dopamine and the Olympic Reward Pathway'}, {'time': 2670.3, 'headline': ' Dopamine and the Pleasure - Pain Balance'}, {'time': 2845.0, 'headline': ' Dopamine - The Secret of Sex and Reproduction'}, {'time': 457.1, 'headline': ' The Subconscious Circuits for Sugar Perception'}, {'time': 3128.7, 'headline': ' So this experiment is so crucial . '}, {'time': 3131.5, 'headline': ' Post Injective Effect - What Is It?'}, {'time': 3172.8, 'headline': ' Neuro pods , Sells Dura and the Neuronal Pathways'}, {'time': 3272.2, 'headline': ' The Sweet Taste of Sugary Foods'}, {'time': 3691.6, 'headline': ' The Pathway of the quickest source of fuel for the brain and the rest of the nervous'}, {'time': 3705.3, 'headline': ' The Glycemic Index and the Neuronal Circuits'}, {'time': 4113.0, 'headline': ' Drug Cocaine Causes Sharp Increases in Dopamine , Within the Brain'}, {'time': 4167.0, 'headline': ' The New Evidence for Type 2 Diabetes'}, {'time': 4478.6, 'headline': ' Is There a Conversion of Artificial Sweeteners?'}, {'time': 4749.8, 'headline': ' Is That a Fair Experiment Or Is It Not Our Natural World Experi'}, {'time': 4781.2, 'headline': ' ADHD - Is It Good For ADHD?'}, {'time': 5117.9, 'headline': ' Sugar Consumption and Highly Refined Sugar Consumption As Potentially Bad For ADHD'}, {'time': 5157.7, 'headline': ' The Importance of Omega - 3 Fatty Acids in ADHD'}, {'time': 5624.8, 'headline': ' Reduce Sugar Cravings - Is It Safe?'}, {'time': 5696.6, 'headline': ' The Secret of the Blood Sugar Monitor'}, {'time': 5894.4, 'headline': ' Lemon Juice and Water Lime Juice and Water Can Decrease Blood Glucos'}, {'time': 5963.8, 'headline': ' Changes in PH . '}, {'time': 5999.2, 'headline': ' Lemon Juice For Lime Juice - Is It a Good Tool For Control'}, {'time': 6214.6, 'headline': ' The Best Way to Lower Blood Glucose Is Toxic Cinnamon'}, {'time': 6459.4, 'headline': ' Increasing Blood Glucose Through Berberine'}, {'time': 6656.8, 'headline': ' The Importance of Sleep in Controlling Sugar Cravings'}, {'time': 7026.1, 'headline': ' The Government Lab Podcast - Part 2'},{'time':7131.0,'headline':''}]
print(len(podcastDict))

42


In [ ]:
similarity(sampleDict,podcastDict)

42
41
24
42
22
[0.00858365487908698, -0.03663562142349477, 0.023802718538608755, 0.3222369410402331, 0.23552027636146064, 0.18277545741256515, 0.2220763300196691, 0.1554072187823394, 0.18079505840191515, 0.2119919067235504, 0.1697013992916716, 0.2041831949576802, 0.2520646237612392, 0.12122194147614966, 0.1385259891120696, 0.19268119723848373, 0.11395042751768532, 0.16156382719077958, 0.045797895242661, 0.24653257762797684, 0.22293280916035793, 0.15189795490407798]
0.0839906613861135


## 0.5 dele spon

In [ ]:
podcastDict = [{'time': 0.1, 'headline': ' The Sweet Taste of Sugar and How It Impacts Your Brain and Body'}, {'time': 121.0, 'headline': ' The Brain Body Contract - Live Event May 1st'}, {'time': 165.7, 'headline': ' The Huberman Lab Podcast '}, {'time': 675.9, 'headline': ' Glucose is Complete Blood Sugar'}, {'time': 718.1, 'headline': ' Diabetes - Is It a Good Idea to Avoid Diabetes?'}, {'time': 875.4, 'headline': " Ketogenic , I'm a Ketoholic I'm a Ke"}, {'time': 912.4, 'headline': ' Orientation Tuning in the Visual System'}, {'time': 1000.6, 'headline': ' Is It Dependent On Blood Glucose Level?'}, {'time': 1249.9, 'headline': ' Glucose is the Preferable Source of Fuel for the Nervous System'}, {'time': 1595.4, 'headline': ' The Effects of Fructose in Fruits'}, {'time': 1739.7, 'headline': ' Is Fructose a Good Thing For Your Appetite?'}, {'time': 1962.8, 'headline': ' Sugar Is So Attractive'}, {'time': 2074.3, 'headline': ' The Parallel Pathways in the Brain and Body Control Sugar Cravings'}, {'time': 2360.4, 'headline': ' Neuroimaging Study - When You Inhale Something Sweet , You Taste It'}, {'time': 2454.6, 'headline': ' Dopamine and the Olympic Reward Pathway'}, {'time': 2670.3, 'headline': ' Dopamine and the Pleasure - Pain Balance'}, {'time': 2845.0, 'headline': ' Dopamine - The Secret of Sex and Reproduction'}, {'time': 2922.9, 'headline': ' The Subconscious Circuits for Sugar Perception'}, {'time': 3128.7, 'headline': ' So this experiment is so crucial . '}, {'time': 3131.5, 'headline': ' Post Injective Effect - What Is It?'}, {'time': 3172.8, 'headline': ' Neuro pods , Sells Dura and the Neuronal Pathways'}, {'time': 3272.2, 'headline': ' The Sweet Taste of Sugary Foods'}, {'time': 3691.6, 'headline': ' The Pathway of the quickest source of fuel for the brain and the rest of the nervous'}, {'time': 3705.3, 'headline': ' The Glycemic Index and the Neuronal Circuits'}, {'time': 4113.0, 'headline': ' Drug Cocaine Causes Sharp Increases in Dopamine , Within the Brain'}, {'time': 4167.0, 'headline': ' The New Evidence for Type 2 Diabetes'}, {'time': 4478.6, 'headline': ' Is There a Conversion of Artificial Sweeteners?'}, {'time': 4749.8, 'headline': ' Is That a Fair Experiment Or Is It Not Our Natural World Experi'}, {'time': 4781.2, 'headline': ' ADHD - Is It Good For ADHD?'}, {'time': 5117.9, 'headline': ' Sugar Consumption and Highly Refined Sugar Consumption As Potentially Bad For ADHD'}, {'time': 5157.7, 'headline': ' The Importance of Omega - 3 Fatty Acids in ADHD'}, {'time': 5624.8, 'headline': ' Reduce Sugar Cravings - Is It Safe?'}, {'time': 5696.6, 'headline': ' The Secret of the Blood Sugar Monitor'}, {'time': 5894.4, 'headline': ' Lemon Juice and Water Lime Juice and Water Can Decrease Blood Glucos'}, {'time': 5963.8, 'headline': ' Changes in PH . '}, {'time': 5999.2, 'headline': ' Lemon Juice For Lime Juice - Is It a Good Tool For Control'}, {'time': 6214.6, 'headline': ' The Best Way to Lower Blood Glucose Is Toxic Cinnamon'}, {'time': 6459.4, 'headline': ' Increasing Blood Glucose Through Berberine'}, {'time': 6656.8, 'headline': ' The Importance of Sleep in Controlling Sugar Cravings'}, {'time': 7026.1, 'headline': ' The Government Lab Podcast - Part 2'},{'time':7131.0,'headline':''}]
print(len(podcastDict))

41


In [ ]:
similarity(sampleDict,podcastDict)

41
40
24
41
22
[0.03208254699798035, -0.007362916177742531, 0.049303612174916765, 0.28769582758737977, 0.16150883801742902, 0.18277545741256515, 0.15464714737217303, 0.14497188497058225, 0.28556287122020313, 0.16739625052897644, 0.2567469354040395, 0.2041831949576802, 0.2520646237612392, 0.12122194147614966, 0.1385259891120696, 0.19268119723848373, 0.11395042751768532, 0.16156382719077958, 0.045797895242661, 0.24653257762797684, 0.22293280916035793, 0.15189795490407798]
0.08699221691945522


In [ ]:
headline = [' The Sweet Taste of Sugar and How It Impacts Your Brain and Body', ' The Brain Body Contract - Live Event May 1st', ' The Nootropics Podcast - The Best of the Best', ' The Effects of Sugar on the Nervous System', ' Glucose is Complete Blood Sugar', ' Diabetes - Is It a Good Idea to Avoid Diabetes?', " Ketogenic , I'm a Ketoholic I'm a Ke", ' Orientation Tuning in the Visual System', ' Is It Dependent On Blood Glucose Level?', ' Glucose is the Preferable Source of Fuel for the Nervous System', ' Is Fructose a Good Thing for Weight Loss?', ' Dopamine and the Pleasure Pain - The Secret of the Mood and', ' The Preference for Sweet Taste in Foods', ' The Secret of the Ketogenic Diet',' Drug Cocaine Causes Sharp Increases in Dopamine , Within the Brain', ' The New Evidence for Type 2 Diabetes', ' Is There a Conversion of Artificial Sweeteners?', ' Is That a Fair Experiment Or Is It Not Our Natural World Experi', ' ADHD - Is Sugar Consumption a Good Idea?', ' Sugar , Consumption of Sugar - Sugar Beverages and', ' Can Glutamine Reduce Sugar Cravings?', ' The Secret of Blood Glucose Monitoring', ' Lemon Juice and Lime Juice Can Reduce Blood Sugar', ' The Best Way to Lower Blood Glucose Is Toxic Cinnamon', ' Increasing Blood Sugar Through Berberine', ' The Role of Sleep in Controlling Sugar Cravings', ' The Government Lab Podcast - Is It Worth It?']


In [ ]:
for h in headline:
  print(h)

 The Sweet Taste of Sugar and How It Impacts Your Brain and Body
 The Brain Body Contract - Live Event May 1st
 The Nootropics Podcast - The Best of the Best
 The Effects of Sugar on the Nervous System
 Glucose is Complete Blood Sugar
 Diabetes - Is It a Good Idea to Avoid Diabetes?
 Ketogenic , I'm a Ketoholic I'm a Ke
 Orientation Tuning in the Visual System
 Is It Dependent On Blood Glucose Level?
 Glucose is the Preferable Source of Fuel for the Nervous System
 Is Fructose a Good Thing for Weight Loss?
 Dopamine and the Pleasure Pain - The Secret of the Mood and
 The Preference for Sweet Taste in Foods
 The Secret of the Ketogenic Diet
 Drug Cocaine Causes Sharp Increases in Dopamine , Within the Brain
 The New Evidence for Type 2 Diabetes
 Is There a Conversion of Artificial Sweeteners?
 Is That a Fair Experiment Or Is It Not Our Natural World Experi
 ADHD - Is Sugar Consumption a Good Idea?
 Sugar , Consumption of Sugar - Sugar Beverages and
 Can Glutamine Reduce Sugar Cravings?
